# Synthetic dataset for the end2end approach

Due to troubles with the dataset of original/highlighted simplified dataset, the decision was made to create a synthetic dataset to train the adapters.

After running the notebook create_synthetic_dataset.ipynb, you can use this notebook to create training and test files for the end2end approach.

(You may find more details in the paper.)

In [3]:
import json

NEW_INSTRUCTION = """
Du erhältst einen Text. Deine Aufgabe ist es, diesen Text zu vereinfachen und mit Markdown zu annotieren.
- Zuerst bereinige den Originaltext: Entferne alle Links, merkwürdige Formatierungen oder andere Artefakte aus der Vorverarbeitung. Es soll nur noch sauberer Text übrig bleiben, aber nichts umgeschrieben werden.
- Nun erstelle aus diesem sauberen Text einen entsprechenden vereinfachten Text.
- Der Text soll für Menschen mit kognitiven Beeinträchtigungen vereinfacht sein: Verwende sehr einfaches Vokabular, einfache Grammatik und wirklich sehr kurze Sätze.
- Jeder Satz sollte in einer neuen Zeile stehen.
- Annotiere Überschriften im Text mit Markdown (#, ## oder ###).
- Markiere alle wichtigen Wörter oder Phrasen fett (**so**).
- Halte dich strikt an das Markdown-Format und füge nur die Markdown-Annotationen hinzu.
- Gib keine Erklärungen, Kommentare oder zusätzlichen Text aus.
""".strip()


def load_jsonl(path):
    with open(path, "r", encoding="utf-8") as f:
        return [json.loads(line) for line in f]


def write_jsonl(path, items):
    with open(path, "w", encoding="utf-8") as f:
        for item in items:
            f.write(json.dumps(item, ensure_ascii=False) + "\n")


def build_combined_file(
    simplifier_path,
    highlighter_path,
    output_path
):
    simplifier = load_jsonl(simplifier_path)
    highlighter = load_jsonl(highlighter_path)

    if len(simplifier) != len(highlighter):
        raise ValueError("Files do not have the same number of entries!")

    combined = []

    for s, h in zip(simplifier, highlighter):
        new_record = {
            "id": s.get("id"),
            "instruction": NEW_INSTRUCTION,
            "input": s.get("input"),
            "output": h.get("output"),
        }

        # include metadata if present
        if "metadata" in s:
            new_record["metadata"] = s["metadata"]

        combined.append(new_record)

    write_jsonl(output_path, combined)
    print(f"Done. Wrote {len(combined)} entries to {output_path}.")


if __name__ == "__main__":
    build_combined_file(    
        simplifier_path="simplifier_val_part2.jsonl",
        highlighter_path="highlighter_val_part2.jsonl",
        output_path="end2end_val_part2.jsonl")


Done. Wrote 6 entries to end2end_val_part2.jsonl.
